In [1]:
from IPython.display import clear_output
import os

In [5]:
!pip install langchain langchain-community langchain-core langchain-openai httpx==0.27.2 langchain-anthropic langchain-google-genai
clear_output()

In [6]:
from typing import List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain_anthropic import  Anthropic
from langchain_core.language_models import BaseLanguageModel
from langchain_core.outputs import LLMResult
from langchain.schema.messages import AIMessage, BaseMessage
from openai import OpenAIError, AuthenticationError
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai.chat_models.base import BaseChatOpenAI

In [7]:
def deepseek_ai(
    model: str = "deepseek-chat",
    temperature: float = 0.5,
    max_tokens: int = 1024,
    **kwargs
) -> BaseChatOpenAI:
    """
    Crea una instancia de Deepseek configurada para Langchain.

    Args:
        model (str): Nombre del modelo a usar
        temperature (float): Creatividad de las respuestas (0-1)
        max_tokens (int): Máximo de tokens en la respuesta
        **kwargs: Parámetros adicionales

    Returns:
        BaseChatOpenAI: Instancia configurada de Deepseek
    """
    return BaseChatOpenAI(
        model=model,
        openai_api_key=os.environ.get("DEEPSEEK_API_KEY"),
        openai_api_base="https://api.deepseek.com",
        temperature=temperature,
        max_tokens=max_tokens,
        **kwargs
    )

In [8]:
def get_fallback_llm(models: List[Dict[str, Any]], temperature: float = 0.5):
    """
    Devuelve una instancia de LLM configurada con el primer modelo funcional de la lista.

    Args:
        models (List[Dict[str, Any]]): Lista de modelos configurados con:
            - "env_var": Variable de entorno para la API key.
            - "api_key": Clave API para el modelo.
            - "class": Clase del modelo (por ejemplo, ChatOpenAI).
            - "params": Parámetros adicionales para inicializar el modelo.
        temperature (float): Temperatura para los modelos.

    Returns:
        BaseLanguageModel: Instancia del primer modelo funcional.

    Raises:
        Exception: Si todos los modelos fallan.
    """
    for model_config in models:
        try:
            # Configurar la API key en la variable de entorno
            os.environ[model_config["env_var"]] = model_config["api_key"]

            # Instanciar el modelo
            llm = model_config["class"](temperature=temperature, **model_config.get("params", {}))
            llm.predict("Ping test")  # Verifica que la API key es válida
            # Si la instancia es exitosa, devolverla
            return llm
        except AuthenticationError as auth_err:
            print(f"Error de autenticación con {model_config['class'].__name__}: {auth_err}")
        except Exception as e:
            print(f"Error al inicializar el modelo {model_config['class'].__name__}: {e}")

    # Si todos los modelos fallan, lanzar excepción
    raise Exception("No se pudo inicializar ningún modelo LLM debido a errores de autenticación u otros fallos.")



In [58]:
with open("/content/api_key_openai.txt") as archivo:
  apikey_gpt = archivo.read()
with open("/content/api_key_claude.txt") as archivo:
  apikey_clude = archivo.read()
with open("/content/api_key_gemini.txt") as archivo:
  apikey_gemini = archivo.read()
with open("/content/api_key_deepseek.txt") as archivo:
  apikey_deepseek = archivo.read()

In [59]:
models = [
    # openAI GPT-4
     {
        "env_var": "OPENAI_API_KEY",
        "api_key": apikey_gpt,
        "class": ChatOpenAI,
        "params": {"model_name": "gpt-4"}
    },
    # Anthropic - Claude
    {
        "env_var": "ANTHROPIC_API_KEY",
        "api_key": apikey_clude,
        "class": Anthropic,
        "params": {"model": "claude-2"}
    },
    # Google - Gemini Pro
    {
        "env_var": "GOOGLE_API_KEY",
        "api_key": apikey_gemini,
        "class": ChatGoogleGenerativeAI,
        "params": {"model": "gemini-pro"}
    },
    # DeepSeek - r1
    {
        "env_var": "DEEPSEEK_API_KEY",
        "api_key": apikey_deepseek,
        "class": deepseek_ai,
        "params": {
            "model": "deepseek-reasoner",
            "max_tokens": 2048
        }
    }
]

In [60]:
llm = get_fallback_llm(models, temperature=0.5)

In [61]:
print("Respuesta: "+llm.predict("¿Cuál es la capital de Francia?"))

Respuesta: La capital de Francia es París.
